### Imports

In [11]:
import nest_asyncio; nest_asyncio.apply()
import re

import json
import os
from dotenv import load_dotenv
load_dotenv()

True

### Function that sends questions to GPT

In [12]:
from openai import OpenAI
client = OpenAI()

model="gpt-4o"
temp = 0
system_message = r"You're an expert in answering quiz questions about phonology."

def pingGPT(prompt, model=model):
   # Creating a message as required by the API
   messages = [{"role":"system", "content": system_message},
               {"role": "user", "content": prompt}]
  
   # Calling the ChatCompletion API
   response = client.chat.completions.create(
       model=model,
       messages=messages,
       temperature=temp,
   )

   # Returning the extracted response
   return response.choices[0].message.content

def assemble_prompt(question):
   context = f"Context information is below.\n----------------\n{question['correct_text_chunk']}\nGiven the context information and not prior knowledge, answer the question. Only answer with the letter and text of the answer which you think is correct:\n"
   return(context + question['question']+'\n' + '\n'.join(question['answers']))

In [13]:
LIST_OF_QUIZZES = ["oracle/JSON_oracles/"+f for f in os.listdir("./oracle/JSON_oracles/") if f.endswith('.json')]

print(LIST_OF_QUIZZES)
print("number of quizzes:", len(LIST_OF_QUIZZES))

['oracle/JSON_oracles/oracle-itembank-2.json', 'oracle/JSON_oracles/oracle-itembank-3.json', 'oracle/JSON_oracles/oracle-itembank-4.json', 'oracle/JSON_oracles/oracle-itembank-5.json', 'oracle/JSON_oracles/oracle-itembank-9.json']
number of quizzes: 5


### Open an itembank, send the questions to raw GPT-4o, obtain and save the responses

In [14]:
# save total number of questions and correct answers separately for my own quizzes and GPT-made
total_nquestions = 0
total_ncorrects = 0

itembank_reports = [] # to write the itembank number and number of correct answers.
itembank_reports.append(f"Model: {model}")
itembank_reports.append(f"Temperature: {temp}")
itembank_reports.append(f"System message: {system_message}\n")

for run_n in range(3): # make three passes over all the quizzes
    print(f"\nPass over all the itembanks number {run_n+1}")
    for doc in LIST_OF_QUIZZES:
        with open(doc, 'r', encoding='UTF-8') as file:
            quiz_JSON = json.load(file)
        
        itembank_number = re.search(r"\d[a|b]?(?:\-\d)?", doc).group()

        print(f"----------processing itembank-{itembank_number}----------")
        
        this_ncorrects = 0 # for correct answers in each separate itembank
        this_nquestions = 0

        # main loop that sends requests and saves answers
        for question in quiz_JSON:
            full_prompt = assemble_prompt(question)
            given_answer = pingGPT(full_prompt)  # Get the response from GPT
            question["GPT's response"] = given_answer  # Append the response to the question
            question["correct"] = given_answer==question["correct answer(s)"] # stores whether the answer is correct
            
            # count how many total responses are correct
            if given_answer==question["correct answer(s)"]: 
                total_ncorrects+=1
                this_ncorrects+=1
            this_nquestions+=1
            total_nquestions+=1

        
        itembank_reports.append(f"itembank-{itembank_number}:\t\tCorrect answers {this_ncorrects} / total questions {this_nquestions} * 100 = {round(this_ncorrects/this_nquestions*100, 2)}%")

        # Save the updated JSON data to a new file
        with open(f'./results/oracle_responses/oracle-{itembank_number}_responses.json', 'w') as outfile:
            json.dump(quiz_JSON, outfile, indent=2)
    itembank_reports.append(" ")

itembank_reports.append(f"\nOverall correct answers {total_ncorrects} / total questions {total_nquestions} * 100 = {round(total_ncorrects/total_nquestions*100, 2)}%")
itembank_reports.append("\n------------------------------------------------------------------------\n\n")

# save the prompt and number of correct questions
with open(f"./results/oracle_{model}_results.txt", 'a', encoding='UTF-8') as report_file:
    report_file.write('\n'.join(itembank_reports))


Pass over all the itembanks number 1
----------processing itembank-2----------
----------processing itembank-3----------
----------processing itembank-4----------
----------processing itembank-5----------
----------processing itembank-9----------

Pass over all the itembanks number 2
----------processing itembank-2----------
----------processing itembank-3----------
----------processing itembank-4----------
----------processing itembank-5----------
----------processing itembank-9----------

Pass over all the itembanks number 3
----------processing itembank-2----------
----------processing itembank-3----------
----------processing itembank-4----------
----------processing itembank-5----------
----------processing itembank-9----------


In [9]:
print(total_ncorrects)

841
